# Notebook

## Setup

In [1]:
In [1]: %load_ext kedro.ipython
import pandas as pd
import numpy as np

[12/05/25 04:18:32] INFO     Using                                                                  __init__.py:270
                             'c:\Root\home\Work\Academic\School\Stanford\Courses\2025-Q4-Autumn\CS                 
                             238 - Decision Making under Uncertainty\Projects\Final                                
                             Project\.venv\Lib\site-packages\kedro\framework\project\rich_logging.y                
                             ml' as logging configuration.                                                         

                    INFO     Registered line magic '%reload_kedro'                                   __init__.py:63

                    INFO     Registered line magic '%load_node'                                      __init__.py:65

                    INFO     Resolved project path as:                                              __init__.py:180
                             c:\Root\home\Work\Academic\School\Stanford\Courses\2025-Q4-Autumn\CS                  
                             238 - Decision Making under Uncertainty\Projects\Final Project.                       
                             To set a different path, run '%reload_kedro <project_root>'                           

[12/05/25 04:18:34] INFO     Kedro project CS 238                                                   __init__.py:146

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:147
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        __init__.py:153

## Load Datasets

In [2]:
alarms = catalog.load("clean_alarms")
status = catalog.load("clean_station_overviews")
# sessions = catalog.load("clean_charging_sessions")

sn_map = catalog.load("map_serial_number")
# ss_map = catalog.load("map_station_status")
# ns_map = catalog.load("map_network_status")
# a_map = catalog.load("map_alarm")

domain_alarms = catalog.load("domain_alarms")
domain_status = catalog.load("domain_status")
domain_observations = catalog.load("domain_observations")

                    INFO     Loading data from clean_alarms (ParquetDataset)...                data_catalog.py:1046

                    INFO     Loading data from clean_station_overviews (ParquetDataset)...     data_catalog.py:1046

                    INFO     Loading data from map_serial_number (ParquetDataset)...           data_catalog.py:1046

                    INFO     Loading data from domain_alarms (ParquetDataset)...               data_catalog.py:1046

                    INFO     Loading data from domain_status (ParquetDataset)...               data_catalog.py:1046

                    INFO     Loading data from domain_observations (ParquetDataset)...         data_catalog.py:1046

## Hidden Markov Model (HMM)

In [3]:
domain_observations.observation.cat.categories


Index(['('Available', 'Reachable')', '('Faulted', 'Reachable')',
       '('Unavailable', 'Reachable')', '('Unavailable', '⊥')',
       '('Unreachable', 'Reachable')', '('Unreachable', 'Unreachable')',
       'Boot up', 'Bootup Due to POWER ON', 'Bootup Due to SOFT RESET',
       'Bootup Due to SWITCH', 'Bootup Due to WATCHDOG',
       'Circuit Sharing Current Reduced', 'Circuit Sharing Current Restored',
       'Data Partition Full', 'Earth Fault Station In Service',
       'Earth Fault Station Out Of Service', 'Fault Cleared', 'GFCI Hard Trip',
       'IP Mismatch Detected', 'Maintenance Required', 'Manual Intervention',
       'Over Current Hard Trip Detected', 'Pilot Unreachable (18)',
       'Pilot current level exceeded', 'Powered Off', 'RFID Update Failed',
       'Reachable', 'Relay Stuck Close', 'Station Not Activated',
       'Unknown RFID', 'Unreachable'],
      dtype='object')

In [ ]:
# Num of hidden states (num functional states * num network states * num events)
N = 3 * 2 * 7
print(f"Number of hidden states: N = {N}")

# Num of observation symbols
K = domain_observations.observation.cat.codes.nunique()
print(f"Number of observation symbols: K = {K}")

# Base observation format


Number of hidden states: N = 42
Number of observation symbols: K = 31


### Pomegranate Implementation

In [5]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import random
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.masked import as_masked_tensor
from pomegranate.hmm import DenseHMM
from pomegranate.distributions import Categorical

In [6]:
num_states = N
num_observation_symbols = K

# torch_device = torch.device("cpu")
torch_device = torch.device("cuda")

# --- Step 1: Define Emission Distributions ---
# We create a list of n_observation_symbols distinct Categorical distributions.
# We initialize them with random probabilities so the model has
# something to start with before training (EM algorithm).
dists = []
for _ in range(num_states):
    # Create random probabilities for the 31 symbols
    init_probs = torch.rand(1, num_observation_symbols)
    # Normalize so they sum to 1.0
    init_probs = init_probs / init_probs.sum(dim=1, keepdim=True)

    # Create the distribution object
    dists.append(Categorical(probs=init_probs).to(torch_device))
[d.probs for d in dists]

[12/05/25 04:18:39] WARNING  c:\Root\home\Work\Academic\School\Stanford\Courses\2025-Q4-Autumn\CS   ]8;id=603115;file://C:\Users\btone\AppData\Roaming\uv\python\cpython-3.13.5-windows-x86_64-none\Lib\warnings.py\warnings.py]8;;\:]8;id=695830;file://C:\Users\btone\AppData\Roaming\uv\python\cpython-3.13.5-windows-x86_64-none\Lib\warnings.py#110\110]8;;\
                             238 - Decision Making under Uncertainty\Projects\Final                                
                             Project\.venv\Lib\site-packages\torch\nn\modules\module.py:1357:                      
                             UserWarning: expandable_segments not supported on this platform                       
                             (Triggered internally at                                                              
                             C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocator                
                             Config.h:35.)                                                                         
                               return t.to(                                                                        
                                                                                                                   


[
    Parameter containing:
tensor([[0.0187, 0.0504, 0.0127, 0.0367, 0.0213, 0.0311, 0.0604, 0.0093, 0.0589,
         0.0150, 0.0039, 0.0141, 0.0118, 0.0343, 0.0209, 0.0433, 0.0193, 0.0609,
         0.0621, 0.0196, 0.0515, 0.0564, 0.0608, 0.0272, 0.0106, 0.0106, 0.0186,
         0.0071, 0.0299, 0.0613, 0.0615]], device='cuda:0'),
    Parameter containing:
tensor([[0.0456, 0.0154, 0.0379, 0.0374, 0.0138, 0.0184, 0.0218, 0.0178, 0.0407,
         0.0225, 0.0474, 0.0384, 0.0509, 0.0486, 0.0457, 0.0349, 0.0499, 0.0521,
         0.0402, 0.0436, 0.0528, 0.0222, 0.0188, 0.0394, 0.0127, 0.0007, 0.0253,
         0.0457, 0.0449, 0.0110, 0.0035]], device='cuda:0'),
    Parameter containing:
tensor([[0.0273, 0.0467, 0.0573, 0.0481, 0.0377, 0.0107, 0.0135, 0.0469, 0.0540,
         0.0169, 0.0085, 0.0610, 0.0441, 0.0498, 0.0480, 0.0144, 0.0342, 0.0184,
         0.0548, 0.0214, 0.0578, 0.0083, 0.0008, 0.0024, 0.0369, 0.0099, 0.0514,
         0.0376, 0.0445, 0.0152, 0.0216]], device='cuda:0'),
    Par

In [8]:
dists[0].probs.shape

torch.Size([1, 31])

In [ ]:
# --- Step 2: Prepare Observation Sequences ---
# Convert observation sequences to torch tensors
num_batches = int(len(o_base) * 0.8)  # Use 80% of data for training
o = [torch.tensor(x, dtype=torch.long) for x in o_base]
o_train = o[:num_batches]
# o_train = random.sample(o, num_batches)

# Pad sequences to create a batch tensor
padded_o = pad_sequence(o_train, batch_first=True, padding_value=-1)

# Create a mask to ignore padding during training
mask = padded_o != -1

# Convert to masked tensor
masked_o = as_masked_tensor(padded_o, mask)

# Add an extra dimension to match expected input shape (batch_size, seq_length, 1)
X_train = masked_o.unsqueeze(-1)
print(f"Shape of training data: {X_train.shape}")

In [ ]:
# --- Step 3: Initialize the HMM ---
# We pass the list of distributions to the DenseHMM.
# 'verbose=True' helps you see the improvement during training.
model = DenseHMM(dists, verbose=True).to(torch_device)

In [ ]:
# --- Step 4: Train the HMM ---
model.fit(X_train.to(torch_device))

In [ ]:
# Save model
torch.save(model, "hmm_model.pt")

In [ ]:
# Get the Observation Matrix (State x Symbol)
# We stack them to get a nice (42, 31) tensor
obs_matrix = torch.stack([d.probs for d in model.distributions])
print("Observation Matrix (State x Symbol):")
obs_matrix.round(decimals=3)

In [ ]:
# Get the Transition Matrix (State x State)
trans_matrix = torch.exp(model.edges)
print("Transition Matrix (State x State):")
trans_matrix.round(decimals=3)

In [ ]:
pd.Series(domain_observations.observation.cat.categories)